In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import itertools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e9/sample_submission.csv
/kaggle/input/playground-series-s5e9/train.csv
/kaggle/input/playground-series-s5e9/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s5e9/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e9/test.csv")
X = train.drop(["id","BeatsPerMinute"], axis = 1).values
y = train["BeatsPerMinute"].values
test_data = test.drop("id", axis = 1).values

In [3]:
print(train.describe())
train.drop("id",axis = 1).corr()

                  id    RhythmScore  AudioLoudness   VocalContent  \
count  524164.000000  524164.000000  524164.000000  524164.000000   
mean   262081.500000       0.632843      -8.379014       0.074443   
std    151313.257586       0.156899       4.616221       0.049939   
min         0.000000       0.076900     -27.509725       0.023500   
25%    131040.750000       0.515850     -11.551933       0.023500   
50%    262081.500000       0.634686      -8.252499       0.066425   
75%    393122.250000       0.739179      -4.912298       0.107343   
max    524163.000000       0.975000      -1.357000       0.256401   

       AcousticQuality  InstrumentalScore  LivePerformanceLikelihood  \
count    524164.000000      524164.000000              524164.000000   
mean          0.262913           0.117690                   0.178398   
std           0.223120           0.131845                   0.118186   
min           0.000005           0.000001                   0.024300   
25%           0.06

,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
RhythmScore,1.000000,-0.017385,0.008737,0.028203,0.010992,0.031535,-0.008946,-0.017640,-0.147350,0.005440
AudioLoudness,-0.017385,1.000000,-0.009357,-0.013019,0.003772,-0.021620,-0.027305,0.004510,0.192411,-0.003327
VocalContent,0.008737,-0.009357,1.000000,0.009370,-0.017739,-0.120294,0.058675,0.018179,0.016124,0.004876
AcousticQuality,0.028203,-0.013019,0.009370,1.000000,0.003247,0.009678,0.037047,-0.022524,-0.418184,-0.000820
InstrumentalScore,0.010992,0.003772,-0.017739,0.003247,1.000000,-0.002403,0.004738,0.009340,-0.019834,0.001900
LivePerformanceLikelihood,0.031535,-0.021620,-0.120294,0.009678,-0.002403,1.000000,0.015461,0.005179,-0.266256,0.003471
MoodScore,-0.008946,-0.027305,0.058675,0.037047,0.004738,0.015461,1.000000,-0.014803,-0.241713,0.007059
TrackDurationMs,-0.017640,0.004510,0.018179,-0.022524,0.009340,0.005179,-0.014803,1.000000,0.048597,0.006637
Energy,-0.147350,0.192411,0.016124,-0.418184,-0.019834,-0.266256,-0.241713,0.048597,1.000000,-0.004375
BeatsPerMinute,0.005440,-0.003327,0.004876,-0.000820,0.001900,0.003471,0.007059,0.006637,-0.004375,1.000000


In [4]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor


In [5]:
def featurepreparation(data, train = True):
    if train:
        features = data.drop(["id", "BeatsPerMinute"], axis = 1).columns
        new_data = data.drop(["id", "BeatsPerMinute"], axis = 1).copy()
        new_data1 = pd.DataFrame()
    else:
        features = data.drop(["id"], axis = 1).columns
        new_data = data.drop(["id"], axis = 1).copy()
        new_data1 = pd.DataFrame()
    for feature1, feature2 in itertools.combinations(features, 2):
        new_var = "{}_x_{}".format(feature1,feature2)
        new_data[new_var] = new_data[feature1] * new_data[feature2]
    for feature1 in features:
        for feature2 in features:
            if feature1 != feature2:
                new_var = "{}_per_{}".format(feature1, feature2)
                new_data1[new_var] = new_data[feature1]/(np.abs(new_data[feature2]))
    new_data = pd.concat([new_data, new_data1], axis = 1)
    return new_data
new_X_train = featurepreparation(train)
new_X_test = featurepreparation(test, train = False)

In [6]:
# Random Forest
#parameters = {'max_features': [int(x) for x in np.logspace(np.log10(1),np.log10(50), num = 5)],
#             'min_samples_leaf': [int(x) for x in np.logspace(np.log10(1), np.log10(256), num = 100)],
#              #randint
#             'min_samples_split': [int(x) for x in np.logspace(np.log10(2), np.log10(256), num = 100)],
#             'max_depth': [3,5,7]}
#rf_regr = RandomForestRegressor(n_estimators = 50)
#CV = RandomizedSearchCV(estimator = rf_regr, param_distributions = parameters, n_iter = 15, cv = 2)
#CV.fit(new_X_train, y)
#print(CV.best_estimator_)
#print(CV.best_score_)

In [ ]:
rf_model = RandomForestRegressor(n_estimators = 800, max_depth = 7, max_features=49, 
                                min_samples_split=13)
rf_model.fit(new_X_train, y)
predictions = rf_model.predict(new_X_test)

output = pd.DataFrame({'id': test.id, 'BeatsPerMinute': predictions})
output.to_csv('submission_rf_new.csv', index=False)
print("Your submission was successfully saved!")